# Scraping NFL Data from MasseyRatings.com

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
URLS = [
    "https://www.masseyratings.com/scores.php?s=nfl2019",
    "https://www.masseyratings.com/scores.php?s=nfl2018",
    "https://www.masseyratings.com/scores.php?s=nfl2017",
    "https://www.masseyratings.com/scores.php?s=nfl2016",
    "https://www.masseyratings.com/scores.php?s=nfl2015",
    "https://www.masseyratings.com/scores.php?s=nfl2014",
    "https://www.masseyratings.com/scores.php?s=nfl2013",
    "https://www.masseyratings.com/scores.php?s=nfl2012",
    "https://www.masseyratings.com/scores.php?s=nfl2011",
    "https://www.masseyratings.com/scores.php?s=nfl2010",
    "https://www.masseyratings.com/scores.php?s=nfl2009",
    "https://www.masseyratings.com/scores.php?s=nfl2008",
    "https://www.masseyratings.com/scores.php?s=nfl2007",
    "https://www.masseyratings.com/scores.php?s=nfl2006",
    "https://www.masseyratings.com/scores.php?s=nfl2005",
    "https://www.masseyratings.com/scores.php?s=nfl2004",
    "https://www.masseyratings.com/scores.php?s=nfl2003",
    "https://www.masseyratings.com/scores.php?s=nfl2002",
    "https://www.masseyratings.com/scores.php?s=nfl2001",
    "https://www.masseyratings.com/scores.php?s=nfl2000",
    "https://www.masseyratings.com/scores.php?s=nfl1999",
    "https://www.masseyratings.com/scores.php?s=nfl1998",
    "https://www.masseyratings.com/scores.php?s=nfl1997",
    "https://www.masseyratings.com/scores.php?s=nfl1996",
    "https://www.masseyratings.com/scores.php?s=nfl1995",
    "https://www.masseyratings.com/scores.php?s=nfl1994",
    "https://www.masseyratings.com/scores.php?s=nfl1993",
    "https://www.masseyratings.com/scores.php?s=nfl1992",
    "https://www.masseyratings.com/scores.php?s=nfl1991",
    "https://www.masseyratings.com/scores.php?s=nfl1990"
]
base = "https://www.masseyratings.com/"
ending = "&all=1&format=1"
output_folder = "nfl_data/"

In [3]:
def create_D(raw):
    data = np.array([[int(entry) for entry in line] for line in [[wd.strip() for wd in line.split(",")] for line in raw][:-1]])[:,[2,4,5,7]]
    data[:,0] -= 1
    data[:,2] -= 1
    n = np.amax(data[:,[0,2]])+1
    D = np.zeros(shape=(n,n), dtype=int)
    for game in data:
        team1,score1,team2,score2 = game
        D[team1,team2] += score1-score2
    return D

def save_D(D, fname):
    n = D.shape[0]
    with open(fname, "w") as fout:
        fout.write(str(n) + "\n")
        for row in D:
            line = ""
            for elem in row:
                line += " " + str(elem)
            fout.write(line + "\n")

In [4]:
# For scraping NFL data
for url in URLS:
    url_category = url.split("=")[-1]
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    links = soup.html.body.find_all('a')
    # Scrape NFC
    page = requests.get(base + links[2]["href"] + ending)
    raw_lines = str(BeautifulSoup(page.content, 'html.parser')).split("\n")
    D = create_D(raw_lines)
    save_D(D, output_folder + url_category + "_nfc")
    # Scrape AFC
    page = requests.get(base + links[3]["href"] + ending)
    raw_lines = str(BeautifulSoup(page.content, 'html.parser')).split("\n")
    D = create_D(raw_lines)
    save_D(D, output_folder + url_category + "_afc")